In [36]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm.session import sessionmaker
from bs4 import BeautifulSoup
import cookielib
import urllib2
import requests
import re

In [37]:
def get_soup(url):
    #request = urllib2.Request(url)
    #response = urllib2.urlopen(url) #request
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html.decode('utf-8', 'ignore'), 'html.parser')
    results = soup.find_all('div', attrs = {'class': 'story-body'})
    print response.status_code
    return results

In [38]:
def scrape_soup(results):

    # initialize lists
    titles = []
    dates = []
    links = []
    full_texts = []
    authors = []
    sections = []

    # scrape results into lists
    for x in results:
        # get link
        link = x.find('a')['href']
        # set regex to eliminate interactive features
        match = re.search('^https://www.nytimes.com/20', link)
        if match:
            links.append(link)

    # resoup it
    for link in links:
        all_p = ''
        cj = cookielib.CookieJar()
        opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
        new_soup = BeautifulSoup(opener.open(link).read().decode('utf-8', 'ignore'), 'html.parser')
        # get the article content
        body = new_soup.find_all('p', attrs = {'class': 'story-body-text story-content'})
        for p in body:
            new_p = p.text.strip()
            all_p = all_p + new_p
        full_texts.append(all_p)

        # get titles
        title = new_soup.find('meta', attrs = {'property': 'og:title'})['content']
        titles.append(title)

        # get authors
        author = new_soup.find('meta', attrs = {'name': 'author'})['content']
        authors.append(author)

        # get sections
        section = new_soup.find('meta', attrs = {'name': 'CG'})['content']
        sections.append(section)

        # get dates
        date = new_soup.find('meta', attrs = {'name': 'pdate'})['content']
        dates.append(date)

    data_dict = {
        'title': titles, 'link': links, 'author': authors, 'body': full_texts,
        'section': sections, 'date': dates
    }
    return data_dict

In [39]:
def make_df(data_dict):
    df = pd.DataFrame(data_dict)
    print "Adding {} rows to staging".format(len(df))
    return df

In [105]:
results = get_soup('http://www.nytimes.com/section/business')
data_dict = scrape_soup(results)
df = make_df(data_dict)

404
Adding 0 rows to staging


In [96]:
df.head(1)

,author,body,date,link,section,title
0,Katie Thomas,President Trump promised some of the nation’s ...,20170131,https://www.nytimes.com/2017/01/31/health/trum...,health,"Trump Vows to Ease Rules for Drug Makers, but ..."


In [97]:
engine = create_engine('postgresql://teresaborcuch@localhost:5433/capstone')

In [98]:
# clear staging
clear_staging_query = 'DELETE FROM nyt_staging *;'

In [99]:
# create sqlalchemy session and clear staging
Session = sessionmaker(bind=engine)
session = Session()

engine.execute(clear_staging_query)
session.commit()

In [100]:
# add dataframe to staging
df.to_sql('nyt_staging', engine, if_exists = 'append', index = False)

In [101]:
move_unique_query = '''
INSERT INTO ny_times (title, date, author, body, link, section)
SELECT title, date, author, body, link, section
FROM nyt_staging
WHERE NOT EXISTS (SELECT title, date, author, body, link, section
FROM ny_times
WHERE ny_times.title = nyt_staging.title);
'''

In [102]:
# run the move_unique_query
engine.execute(move_unique_query)
session.commit()
session.close()

In [22]:
# delete duplicates
del_dup_query = '''
DELETE FROM ny_times WHERE id NOT in 
(SELECT MIN(id) FROM ny_times GROUP BY title);
'''